In [ ]:
import sys
import os
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
sys.path.append('..') 
from data.df_cleaning import df_cleaned_result

df = df_cleaned_result.copy()

# --- REAL WORLD BENCHMARKS ---

# 1. Distance Benchmarks (Approx. Km per migration)
REAL_WORLD_DISTANCE = {
    'Stork': 10000,
    'Crane': 2500,
    'Swallow': 9000,
    'Eagle': 3000,
    'Goose': 4000,
    'Warbler': 2700,
    'Hawk': 9000
}

# 2. Altitude Benchmarks (Approx. Max Meters)

REAL_WORLD_ALTITUDE = {
    'Stork': 4800,
    'Crane': 10000,  
    'Swallow': 2000, 
    'Eagle': 6000,
    'Goose': 8800,   
    'Warbler': 3000,
    'Hawk': 4500
}

# Aggregate Data
df_species_stats = df.groupby('Species').agg(
    Max_Altitude=('Max_Altitude_m', 'max'),
    Avg_Altitude=('Max_Altitude_m', 'mean'),
    Max_Distance=('Flight_Distance_km', 'max'),
    Avg_Distance=('Flight_Distance_km', 'mean')
).reset_index()

# Map Benchmark Data
df_species_stats['Real_World_Distance'] = df_species_stats['Species'].map(REAL_WORLD_DISTANCE).fillna(0)
df_species_stats['Real_World_Altitude'] = df_species_stats['Species'].map(REAL_WORLD_ALTITUDE).fillna(0)

df_species_stats = df_species_stats.sort_values(by='Max_Altitude', ascending=False)
ALL_SPECIES = df_species_stats['Species'].unique().tolist()


# --- 2. APP SETUP ---
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUMEN, dbc.icons.BOOTSTRAP])

app.layout = dbc.Container([
    
    # --- HEADER & DESCRIPTION ---
    dbc.Row([
        dbc.Col([
            html.H2("Species Migration Explorer", className="display-6 text-primary"),
            html.P([
                "This dashboard explores bird migration patterns across seven species using ",
                html.Strong("synthetic data"), ". ",
                "To help evaluate the simulation's realism, we have integrated Real World Benchmarks, based on ornithological records.",
                html.Br(),
                "Use the controls below to compare ", 
                html.Strong("Altitude"), " and ", html.Strong("Flight Distance"), "."
            ], className="lead text-muted"),
        ], width=12)
    ], className="mt-4 mb-4"),
    
    # --- MAIN CONTENT ROW ---
    dbc.Row([
        # --- SIDEBAR ---
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Configuration", className="fw-bold bg-light"),
                dbc.CardBody([
                    
                    # 1. SPECIES FILTER
                    html.Label("Select Species", className="fw-bold text-primary"),
                    dcc.Dropdown(
                        id='species-filter',
                        options=[{'label': s, 'value': s} for s in ALL_SPECIES],
                        value=[], 
                        multi=True, 
                        clearable=True,
                        placeholder="Select species..."
                    ),
                    dbc.Button("Select All Species", id="btn-all-species", color="secondary", outline=True, size="sm", className="mt-2 w-100 mb-3"),
                    
                    html.Hr(),
                    
                    # 2. DATA CATEGORY
                    html.Label("Choose Data Category", className="fw-bold text-primary"),
                    dbc.RadioItems(
                        id='category-selector',
                        options=[
                            {'label': ' Altitude Data (Meters)', 'value': 'Altitude'},
                            {'label': ' Flight Distance (Kilometers)', 'value': 'Distance'}
                        ],
                        value='Altitude', 
                        className="mb-3",
                        inputClassName="me-2"
                    ),

                    html.Hr(),

                    # 3. STATISTIC CHECKLIST
                    html.Label("Select Statistics", className="fw-bold text-primary"),
                    dbc.Checklist(
                        id='statistic-checklist',
                        options=[
                            {'label': ' Maximum', 'value': 'Max'},
                            {'label': ' Average', 'value': 'Avg'},
                            # Real World is added dynamically, but we can default it here too
                        ],
                        value=['Max', 'Avg'], 
                        switch=True, 
                        className="mb-3"
                    ),
                    
                ])
            ], className="mb-4 shadow-sm border-0")
        ], width=12, md=3), 
        
        # --- GRAPH ---
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='main-bar-chart', style={'height': '75vh'}) 
                ], style={'padding': '0'})
            ], className="shadow-sm border-0")
        ], width=12, md=9)
    ]),
], fluid=True, className="bg-light", style={'minHeight': '100vh'})


# --- 3. CALLBACKS ---

@app.callback(
    Output('species-filter', 'value'),
    Input('btn-all-species', 'n_clicks'),
    State('species-filter', 'options')
)
def select_all_species(n_clicks, options):
    if n_clicks:
        return [option['value'] for option in options]
    return dash.no_update

# Callback: Update Options (Real World is now available for BOTH)
@app.callback(
    [Output('statistic-checklist', 'options'),
     Output('statistic-checklist', 'value')],
    Input('category-selector', 'value'),
    State('statistic-checklist', 'value')
)
def update_stat_options(category, current_values):

    options = [
        {'label': ' Maximum', 'value': 'Max'},
        {'label': ' Average', 'value': 'Avg'},
        {'label': ' Real World Benchmark', 'value': 'Real_World'}
    ]

    
    new_values = current_values

    if 'Real_World' not in new_values:
        new_values.append('Real_World')

    return options, new_values


@app.callback(
    Output('main-bar-chart', 'figure'),
    [Input('species-filter', 'value'),
     Input('category-selector', 'value'),   
     Input('statistic-checklist', 'value')] 
)
def update_chart(selected_species, category, selected_stats):
    
    # Empty State
    if not selected_species:
        fig = px.bar()
        fig.update_layout(
            title={
                'text': "Select species from the configuration panel to begin analysis",
                'y': 0.5, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'middle'
            },
            xaxis={'visible': False}, yaxis={'visible': False},
            plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
            font=dict(size=18, color="#6c757d")
        )
        return fig

    # 1. Setup Columns and Labels based on Category
    cols_to_plot = []
    
    if category == 'Altitude':
        y_label = "Altitude (meters)"
        title_prefix = "Altitude Comparison"
       
        color_map = {
            'Max_Altitude': '#0d6efd',   
            'Avg_Altitude': '#aecdf7',    
            'Real_World_Altitude': '#fd7e14' 
        }
        
        # Build Column List
        for stat in selected_stats:
            if stat == 'Real_World':
                cols_to_plot.append('Real_World_Altitude') # <--- KEY CHANGE
            else:
                cols_to_plot.append(f"{stat}_{category}")

    else: # Distance
        y_label = "Flight Distance (km)"
        title_prefix = "Flight Distance Comparison"
        # Define Color Map for Distance
        color_map = {
            'Max_Distance': '#198754',     
            'Avg_Distance': '#a3cfbb',    
            'Real_World_Distance': '#fd7e14' 
        }
        
        # Build Column List
        for stat in selected_stats:
            if stat == 'Real_World':
                cols_to_plot.append('Real_World_Distance')
            else:
                cols_to_plot.append(f"{stat}_{category}")

    # 2. Filter Data
    df_filtered = df_species_stats[df_species_stats['Species'].isin(selected_species)]
    
    # 3. Melt
    valid_cols = [c for c in cols_to_plot if c in df_filtered.columns]
    
    df_melted = df_filtered.melt(
        id_vars='Species', 
        value_vars=valid_cols, 
        var_name='Metric', 
        value_name='Value'
    )
    
    if df_melted.empty:
         return px.bar(title="No data available for the selected options.")

    # 4. Graph
    fig = px.bar(
        df_melted,
        x='Species',
        y='Value',
        color='Metric',
        barmode='group',
        title=f"{title_prefix}",
        template='plotly_white',
        text_auto=',.0f', 
        color_discrete_map=color_map 
    )
    
    fig.update_layout(
        legend_title_text="Metric",
        xaxis_title=None,
        margin=dict(t=60, b=40, l=40, r=180),
        legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02),
        yaxis=dict(title=y_label, tickformat=","),
        transition={'duration': 500}
    )
    
    return fig


if __name__ == '__main__':
    app.run(debug=True, port=8051)

 ## 2.2 RQ2: Which bird species travel the longest or shortest distances on aver
age?
 * Goal: Identify species with extreme migration distances.
 * Visualization: Bar chart showing the average flight distance per species. Users can sort or filter species.
 * Notes: Maybe find interactivity by implementing a slider for distance.

 


 ## 2.3 RQ3: Which species migrate at the highest altitudes?
 * Goal: Compare maximum altitudes reached across species.
 * Visualization: Bar chart or violin plot of max altitudes per species.
 * Notes: Maybe find interactivity by implementing a slider for altitudes.